In [ ]:
# --- FINAL OVERLAY ---
# Overlay both Cell and Nuclei masks on original image for verification

from cellpose.utils import outlines_list

plt.figure(figsize=(12, 12))
plt.imshow(img_rgb)

# Get outlines for cells
outlines_cells = outlines_list(masks_cells)
for o in outlines_cells:
    plt.plot(o[:, 0], o[:, 1], color='r', linewidth=1, label='Cell' if 'Cell' not in plt.gca().get_legend_handles_labels()[1] else "")

# Get outlines for nuclei
outlines_nuclei = outlines_list(masks_nuclei)
for o in outlines_nuclei:
    plt.plot(o[:, 0], o[:, 1], color='cyan', linewidth=1, label='Nucleus' if 'Nucleus' not in plt.gca().get_legend_handles_labels()[1] else "")

plt.title("Combined Segmentation: Red=Cells, Cyan=Nuclei")
plt.axis('off')
plt.legend()
plt.show()

print("Notebook Parts 1 and 2 complete.")
print("Proceed to next steps for quantification logic (counting nuclei inside cell masks).")

In [ ]:
# Initialize Cellpose Nuclei model
model_nuclei = models.Cellpose(gpu=use_gpu, model_type='nuclei')

print("Running Cellpose for nuclei...")
# Run on the single grayscale channel
masks_nuclei, flows_n, styles_n, diams_n = model_nuclei.eval(
    target_nuclei_norm, 
    diameter=None, # Auto-detect diameter
    flow_threshold=None,
    channels=[0, 0] # Grayscale input
)

# Visualize
fig = plt.figure(figsize=(10, 5))
plot.show_segmentation(fig, np.stack([target_nuclei_norm]*3, axis=-1), masks_nuclei, flows_n[0], channels=[0,0])
plt.title("Nuclei Segmentation Results")
plt.show()

print(f"Detected {masks_nuclei.max()} nuclei.")

In [ ]:
# Separate Stains
ihc_hed = color.separate_stains(img_rgb, color.hed_from_rgb)

# Channels: 0=Hematoxylin, 1=Eosin, 2=DAB
hem_channel = ihc_hed[:, :, 0]
dab_channel = ihc_hed[:, :, 2]

# Visualize extraction
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
ax = axes.ravel()

ax[0].imshow(img_rgb)
ax[0].set_title("Original Image")

ax[1].imshow(hem_channel, cmap='gray')
ax[1].set_title("Hematoxylin (Blue/Nuclei)")

ax[2].imshow(dab_channel, cmap='gray')
ax[2].set_title("DAB (Brown/Nuclei?)")

for a in ax:
    a.axis('off')

plt.show()

# --- SELECT TARGET CHANNEL ---
# Ideally, choose the channel with the best contrast for your "brown nuclei".
# If they are truly brown, 'dab_channel' is best.
# If they are blue/purple, 'hem_channel' is best.
# We will assume DAB channel here based on your description, but feel free to switch to 'hem_channel'.
target_nuclei_channel = dab_channel 

# Normalize for Cellpose usually not strictly needed as it auto-normalizes, 
# but passing a clean grayscale image helps.
# Cellpose expects 0-255 or 0-1, but separate_stains returns arbitrary float values often.
# Let's normalize it to 0-255 for visualization consistency and input.
target_nuclei_norm = restoration.denoise_tv_chambolle(target_nuclei_channel, weight=0.1) # Optional denoising
target_nuclei_norm = (target_nuclei_norm - target_nuclei_norm.min()) / (target_nuclei_norm.max() - target_nuclei_norm.min()) * 255
target_nuclei_norm = target_nuclei_norm.astype(np.uint8)

plt.figure()
plt.imshow(target_nuclei_norm, cmap='gray')
plt.title("Processed Channel for Nuclei Segmentation")
plt.axis('off')
plt.show()

## Part 2: Nuclei Segmentation (Color Deconvolution + Cellpose)

Since the nuclei are described as "brown" (likely DAB stained or dense Hematoxylin), simple RGB thresholding might fail. We will use **Color Deconvolution** to separate the stains.
*   We use the **HED (Hematoxylin, Eosin, DAB)** separation matrix.
*   **Hematoxylin:** Stains nuclei blue/purple.
*   **DAB:** Stains brown.
*   **Eosin:** Stains cytoplasm pink.

We will visualize the separated channels and choose the best one (usually the 'DAB' or 'Hematoxylin' channel) to run the Cellpose `nuclei` model.

In [ ]:
# Initialize Cellpose model
model_cyto = models.Cellpose(gpu=use_gpu, model_type='cyto2')

# Define parameters
# Adjust 'diameter' if your cells are significantly larger or smaller
# channels=[1, 2] means: Segment Green channel (none here usually, but fits cytoplasm structure often) or just Red?
# For H&E: Red (channel 0) is often cytoplasm. Blue (channel 2) is nuclei.
# Cellpose channels: [cytoplasm, nuclei]. 
# In RGB: Red=0, Green=1, Blue=2.
# H&E: Redish pink = Muscle. Blue/Purple = Nuclei.
# So we want to segment based on Red channel.
# channels = [1, 2] -> 1=Red (0 in Py), 2=Green (1 in Py). Wait, Cellpose documentation: 
# 0=grayscale, 1=red, 2=green, 3=blue.
# We want to segment cytoplasm (Red) and optionally use Nuclei (Blue) as seed.
channels_cells = [1, 3] # Segment based on Red (1), Nuclei in Blue (3) might help if distinct 

print("Running Cellpose for muscle cells...")
masks_cells, flows, styles, diams = model_cyto.eval(
    img_rgb, 
    diameter=None, # Let Cellpose estimate diameter
    flow_threshold=0.4, 
    channels=channels_cells
)

# Visualize
fig = plt.figure(figsize=(10, 5))
plot.show_segmentation(fig, img_rgb, masks_cells, flows[0], channels=channels_cells)
plt.title("Muscle Cell Segmentation Results")
plt.show()

print(f"Detected {masks_cells.max()} muscle cells.")

## Part 1: Muscle Cell Segmentation (Cellpose)

We use Cellpose to segment the muscle cells. 
*   **Model:** `cyto2` (works well for cytoplasm and general cells).
*   **Target:** Red/Pink areas.
*   **Diameter:** Set approximately based on your visual inspection (e.g., 50-100 pixels). If set to `None`, Cellpose will estimate it.

In [ ]:
# --- CONFIGURATION ---
# Replace this with your actual image path
IMAGE_PATH = "path/to/your/image.tif" 

# Load image
try:
    if os.path.exists(IMAGE_PATH):
        img_rgb = tifffile.imread(IMAGE_PATH)
        # If image has alpha channel, remove it
        if img_rgb.shape[-1] == 4:
            img_rgb = img_rgb[..., :3]
            
        plt.figure(figsize=(10, 10))
        plt.imshow(img_rgb)
        plt.title(f"Original Image: {os.path.basename(IMAGE_PATH)}")
        plt.axis('off')
        plt.show()
    else:
        print(f"Image not found at {IMAGE_PATH}. Please update the IMAGE_PATH variable.")
        # Create a dummy image for demonstration if file doesn't exist
        print("Creating dummy image for demonstration...")
        img_rgb = np.zeros((512, 512, 3), dtype=np.uint8)
        img_rgb[:] = [240, 240, 250] # Background
        # Add red muscle cells
        from skimage.draw import disk
        rr, cc = disk((256, 256), 100)
        img_rgb[rr, cc] = [200, 50, 50] # Red muscle
        # Add brown nuclei
        rr_n, cc_n = disk((256, 256), 15)
        img_rgb[rr_n, cc_n] = [100, 50, 20] # Brown nuclei
        
        plt.figure(figsize=(5, 5))
        plt.imshow(img_rgb)
        plt.title("Dummy Demonstration Image")
        plt.show()

except Exception as e:
    print(f"Error loading image: {e}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from skimage import io, color, restoration
from cellpose import models
from cellpose import plot
import os

# Check for GPU
import torch
use_gpu = torch.cuda.is_available()
print(f"GPU Available: {use_gpu}")

# HE Slide Analysis: Muscle Cell and Nuclei Segmentation

This notebook performs two main tasks on HE stained slide images:
1.  **Muscle Cell Segmentation:** Identifies individual muscle cells (pink/red cytoplasm) using Cellpose.
2.  **Nuclei Segmentation:** Identifies brown nuclei within the cells using Color Deconvolution followed by Cellpose.

## 1. Setup and Imports